### Load the model

In [56]:
import librosa
import numpy as np
import pandas as pd
import os
from keras.models import model_from_json

def getModel():
    json_file = open('models/model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights("models/Emotion_Voice_Detection_Model.h5")
    print("Loaded model and weights from disk")
    return loaded_model
loaded_model = getModel()

### Features from the file

In [58]:
def getEmotionPredictionChunk(f):
    labels = ['female_angry','female_calm','female_fearful','female_happy','female_sad','male_angry','male_calm','male_fearful','male_happy','male_sad']
    X, sample_rate = librosa.load(f, res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
    sample_rate = np.array(sample_rate)
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13),axis=0)
    featurelive = mfccs
    livedf2 = featurelive
    livedf2= pd.DataFrame(data=livedf2)
    livedf2 = livedf2.stack().to_frame().T
    twodim= np.expand_dims(livedf2, axis=2)
    livepreds = loaded_model.predict(twodim, 
                         batch_size=32, 
                         verbose=1)
    livepreds1=livepreds.argmax(axis=1)
    return labels[livepreds1[0]], str(livepreds[0][livepreds1[0]])

In [59]:
predicted, score = getEmotionPredictionChunk('/home/absin/git/sentenceSimilarity/speech/RawData/03-01-07-01-01-01-11.wav')
print('Predicted label - > ' + predicted + ' with score -> ' + score)

1/1 [==============================] - 0s 35ms/step
Predicted label - > male_sad with score -> 0.9076831


### Large files VAD then emotion recognition

### Benchmark

In [52]:
def get_file_metadata(file_name):
    split = file_name.split('-')
    pred = ''        
    if(split[2] == '01'):
        pred = 'neutral'
    elif(split[2] == '02'):
        pred = 'calm'
    elif(split[2] == '03'):
        pred = 'happy'
    elif(split[2] == '04'):
        pred = 'sad'
    elif(split[2] == '05'):
        pred = 'angry'
    elif(split[2] == '06'):
        pred = 'fearful'
    elif(split[2] == '07'):
        pred = 'disgust'
    elif(split[2] == '08'):
        pred = 'surprised'
    return pred

folder_path = '/home/absin/git/sentenceSimilarity/speech/RawData/'
totalCount = 0
successCount = 0   
for f in os.listdir(folder_path):
    try:
        predicted, score = getEmotionPredictionChunk(folder_path + f)
        actual = get_file_metadata(f)
        if len(actual)>0:
            totalCount += 1
            if predicted.split('_')[1] == actual:
                successCount += 1
    except ValueError:
        print("An exception occurred with "+f)
print("Final result: "+str(successCount)+" out of "+str(totalCount))

1/1 [==============================] - 0s 2ms/step


1/1 [==============================] - 0s 2ms/step


1/1 [==============================] - 0s 2ms/step


1/1 [==============================] - 0s 2ms/step
An exception occurred with 03-01-08-01-02-02-11.wav
1/1 [==============================] - 0s 2ms/step


1/1 [==============================] - 0s 2ms/step


1/1 [==============================] - 0s 3ms/step


1/1 [==============================] - 0s 2ms/step


1/1 [==============================] - 0s 2ms/step
An exception occurred with 03-01-02-01-02-02-13.wav
1/1 [==============================] - 0s 2ms/step
Final result: 842 out of 1178
